## Walkthrough of Data Science - Traveler

#### [1] Pre-processing: Assessing and analyzing data, cleaning, transforming and adding new features
#### [2] Learning model: Constructing and testing learning model
#### [3] Post-processing: Creating final predictions


## Recap LAB 01 CODE

In [13]:
##Exploring Traveler data
import pandas as pd
import matplotlib.pyplot as plt
%pylab inline 

print("Reading data...")
train_file = "./traveler/train_users_2.csv"
df_train = pd.read_csv(train_file, header = 0,index_col=None)

test_file = "./traveler/test_users.csv"
df_test = pd.read_csv(test_file, header = 0,index_col=None)

# Combining into one dataset for cleaning
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
print("Reading data...completed")

# Fixing date formats in Pandas - to_datetime
## Change dates to specific format
print("Fixing timestamps...")
df_all['date_account_created'] = pd.to_datetime(df_all['date_account_created'], format='%Y-%m-%d')
df_all['timestamp_first_active'] = pd.to_datetime(df_all['timestamp_first_active'], format='%Y%m%d%H%M%S')
print("Fixing timestamps...completed")

## Removing date_first_booking column
df_all.drop('date_first_booking', axis = 1, inplace = True)
print("Droped date_first_booking column...")

import numpy as np

## Remove outliers function - [1]
def remove_outliers(df, column, min_val, max_val):
    col_values = df[column].values
    df[column] = np.where(np.logical_or(col_values<=min_val, col_values>=max_val), np.NaN, col_values)
    return df

## Fixing age column - [2]
print("Fixing age column...")
df_all = remove_outliers(df = df_all, column = 'age', min_val = 15, max_val = 90)
df_all['age'].fillna(-1, inplace = True)
print("Fixing age column...completed")

# Other column missing value - Fill first_affiliate_tracked column
print("Filling first_affiliate_tracked column...")
df_all['first_affiliate_tracked'].fillna(-1, inplace=True)
print("Filling first_affiliate_tracked column...completed")

df_all.head()

Populating the interactive namespace from numpy and matplotlib
Reading data...
Reading data...completed
Fixing timestamps...
Fixing timestamps...completed
Droped date_first_booking column...
Fixing age column...
Fixing age column...completed
Filling first_affiliate_tracked column...
Filling first_affiliate_tracked column...completed


,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
0,direct,direct,-1,NDF,2010-06-28,untracked,Chrome,Mac Desktop,-unknown-,gxn3p5htnn,en,Web,0,facebook,2009-03-19 04:32:55
1,seo,google,38,NDF,2011-05-25,untracked,Chrome,Mac Desktop,MALE,820tgsjxq7,en,Web,0,facebook,2009-05-23 17:48:09
2,direct,direct,56,US,2010-09-28,untracked,IE,Windows Desktop,FEMALE,4ft3gnwmtx,en,Web,3,basic,2009-06-09 23:12:47
3,direct,direct,42,other,2011-12-05,untracked,Firefox,Mac Desktop,FEMALE,bjjt8pjhuk,en,Web,0,facebook,2009-10-31 06:01:29
4,direct,direct,41,US,2010-09-14,untracked,Chrome,Mac Desktop,-unknown-,87mebub9p4,en,Web,0,basic,2009-12-08 06:11:05


### What Next?
### Aim: Focus on transforming the data and feature extraction
##### Why? To make better prediction learning model.

## Data Transformation and Feature Extraction as a Concept


### Why transformation?
##### Unlike the steps taken during cleaning, which are designed to address problems with the raw data (missing and erroneous values, formatting issues etc.), 
##### Data Transformation steps change the values and/or structure of the data (data transformation) and add additional features (feature extraction).


## Data Transformation methods -

### [1] Bucketing/Binning

##### - the numerical values in a particular column are converted from a continuous series into fixed ranges
##### - example, instead of using the age value of all users, we could place them into buckets such as 15-20 years old, 21-25 years old and so on

### [2] Normalization

### [3] Other Transformations

##### There are unlimited number of ways that the numerical values of a given column can be transformed such that they are more suitable for the algorithm being used.

##### - Logarithm function: This transformation is a commonly used method of dealing with exponential data series (i.e. a column where there are lot of low values and relatively few high values). 

#### [3.1] One Hot Encoding (used for categorical data)
<img src="./images/One_Hot_Encoding.png" height="400" width="500"/>

## Feature Extraction -
##### feature construction and feature selection to add to dataset

### [1] Using Hierarchical Information
 ##### data in dataset represents one level of hierarchy, and extracting other implied levels of that hierarchy will provide the learning model with useful information
 
 ### Eg: Consider date fields,  
     - extracting the day of the week, 
     - the month of the year,
     - the hour of the day, 
   #### could add important information for the algorithm to use
     - Maybe people who create their accounts in summer months are more likely to make a booking in a warmer country. 
     - Maybe people who were first active late at night are more disorganized travelers and are therefore more likely to make a domestic first booking. 
     - Combination of these factors may make the difference (e.g. users first active late at night, in the summer months, on a weekday are more likely to travel to Portugal). 
   #### Note:The point is not to be able to explain why a factor may be important, but to think of as many factors as possible to test, and allow the algorithm to determine what is important and not important.
    
### [2] Adding External Data
 ##### (known as record linkage) data enrichment
 #### Eg: Countries could be enriched with demographic data about the country such as 
          - population, 
          - income per capita or 
          - land area 
   ##### all the factors that may allow the algorithm to draw conclusions across similar groups of countries.
 
 ##### Consider how much more accurately we could predict a first booking country of a user if we could link the data from their TRAVELER profile to data from one of their social media profiles (Facebook, Twitter etc.) or even better, from other Traveler accounts.
 
 
 ### IMPORTANT: The key point here is that it is worth investing time looking for ways to add new and useful data to your existing dataset before moving onto the modeling step. 
 ### Expanding your dataset in this manner will often produce far bigger improvements in prediction accuracy than the choice of algorithm or the tuning of the algorithm parameters.

# Practical: Transforming Categorical Data
### One Hot Enconder method 
   ##### – replacing the categorical fields in the dataset with multiple columns representing one value from each column
   ##### - with Scikit Learn (http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)
   ##### - own function

In [12]:
## Prototype for only gender column  - testing
# User defined One Hot Encoding function
def convert_to_binary(df,column_to_convert):
    categories = list(df[column_to_convert].drop_duplicates())
    print categories
    for category in categories:
        cat_name = str(category).replace(" ", "_").replace("(", "").replace(")", "").replace("/", "_").replace("-", "").lower()
        col_name = column_to_convert[:5] + '_' + cat_name[:10]
        print col_name
        df[col_name] = 0
        df.loc[(df[column_to_convert] == category), col_name] = 1

    return df

# One Hot Encoding
print("One Hot Encoding categorical data...")
#columns_to_convert = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
columns_to_convert = ['gender']
for column in columns_to_convert:
    df_all = convert_to_binary(df=df_all, column_to_convert=column)
    #df_all.drop(column, axis=1, inplace=True)
print("One Hot Encoding categorical data...completed")
df_all.head()

One Hot Encoding categorical data...
['-unknown-', 'MALE', 'FEMALE', 'OTHER']
gende_unknown
gende_male
gende_female
gende_other
One Hot Encoding categorical data...completed


,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active,gende_unknown,gende_male,gende_female,gende_other
0,direct,direct,-1,NDF,2010-06-28,untracked,Chrome,Mac Desktop,-unknown-,gxn3p5htnn,en,Web,0,facebook,2009-03-19 04:32:55,1,0,0,0
1,seo,google,38,NDF,2011-05-25,untracked,Chrome,Mac Desktop,MALE,820tgsjxq7,en,Web,0,facebook,2009-05-23 17:48:09,0,1,0,0
2,direct,direct,56,US,2010-09-28,untracked,IE,Windows Desktop,FEMALE,4ft3gnwmtx,en,Web,3,basic,2009-06-09 23:12:47,0,0,1,0
3,direct,direct,42,other,2011-12-05,untracked,Firefox,Mac Desktop,FEMALE,bjjt8pjhuk,en,Web,0,facebook,2009-10-31 06:01:29,0,0,1,0
4,direct,direct,41,US,2010-09-14,untracked,Chrome,Mac Desktop,-unknown-,87mebub9p4,en,Web,0,basic,2009-12-08 06:11:05,1,0,0,0


In [14]:
def convert_to_binary(df, column_to_convert):
    categories = list(df[column_to_convert].drop_duplicates())

    for category in categories:
        cat_name = str(category).replace(" ", "_").
        replace("(", "").replace(")", "").replace("/", "_").
        replace("-", "").lower()
        col_name = column_to_convert[:5] + '_' + cat_name[:10]
        df[col_name] = 0
        df.loc[(df[column_to_convert] == category), col_name] = 1

    return df

# One Hot Encoding
print("One Hot Encoding categorical data...")
columns_to_convert = ['gender', 'signup_method', 'signup_flow', 
                      'language', 'affiliate_channel', 
                      'affiliate_provider', 'first_affiliate_tracked', 
                      'signup_app', 'first_device_type', 'first_browser']

for column in columns_to_convert:
    df_all = convert_to_binary(df=df_all, column_to_convert=column)
    df_all.drop(column, axis=1, inplace=True)
print("One Hot Encoding categorical data...completed")

One Hot Encoding categorical data...
One Hot Encoding categorical data...completed


In [15]:
df_all.head()

,age,country_destination,date_account_created,id,timestamp_first_active,gende_unknown,gende_male,gende_female,gende_other,signu_facebook,...,first_theworld_b,first_slimbrowse,first_epic,first_stainless,first_googlebot,first_outlook_20,first_icedragon,first_ibrowse,first_nintendo_b,first_uc_browser
0,-1,NDF,2010-06-28,gxn3p5htnn,2009-03-19 04:32:55,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,38,NDF,2011-05-25,820tgsjxq7,2009-05-23 17:48:09,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,56,US,2010-09-28,4ft3gnwmtx,2009-06-09 23:12:47,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,42,other,2011-12-05,bjjt8pjhuk,2009-10-31 06:01:29,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,41,US,2010-09-14,87mebub9p4,2009-12-08 06:11:05,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Creating New Features
##### Since there is not a huge number of columns to work, it limits what new features we can add based on the existing data.
##### However, using two date fields – date_account_created and timestamp_first_active - we create some new features 
##### (Extract all the information out of these two date fields that could potentially differentiate which country someone will make their first booking in.)
## 

In [16]:
# Add new date related fields
print("Adding new fields...")
df_all['day_account_created'] = df_all['date_account_created'].dt.weekday
df_all['month_account_created'] = df_all['date_account_created'].dt.month
df_all['quarter_account_created'] = df_all['date_account_created'].dt.quarter
df_all['year_account_created'] = df_all['date_account_created'].dt.year
df_all['hour_first_active'] = df_all['timestamp_first_active'].dt.hour
df_all['day_first_active'] = df_all['timestamp_first_active'].dt.weekday
df_all['month_first_active'] = df_all['timestamp_first_active'].dt.month
df_all['quarter_first_active'] = df_all['timestamp_first_active'].dt.quarter
df_all['year_first_active'] = df_all['timestamp_first_active'].dt.year
df_all['created_less_active'] = (df_all['date_account_created'] - 
                                 df_all['timestamp_first_active']).dt.days
print("Adding new fields...completed")

# Drop unnecessary columns
print("Droping fields...")
columns_to_drop = ['date_account_created', 
                   'timestamp_first_active', 'date_first_booking', 
                   'country_destination']
for column in columns_to_drop:
    if column in df_all.columns:
        df_all.drop(column, axis=1, inplace=True)
print("Droping fields...completed")

Adding new fields...
Adding new fields...completed
Droping fields...
Droping fields...completed


In [17]:
df_all.head()

,age,id,gende_unknown,gende_male,gende_female,gende_other,signu_facebook,signu_basic,signu_google,signu_weibo,...,day_account_created,month_account_created,quarter_account_created,year_account_created,hour_first_active,day_first_active,month_first_active,quarter_first_active,year_first_active,created_less_active
0,-1,gxn3p5htnn,1,0,0,0,1,0,0,0,...,0,6,2,2010,4,3,3,1,2009,465
1,38,820tgsjxq7,0,1,0,0,1,0,0,0,...,2,5,2,2011,17,5,5,2,2009,731
2,56,4ft3gnwmtx,0,0,1,0,0,1,0,0,...,1,9,3,2010,23,1,6,2,2009,475
3,42,bjjt8pjhuk,0,0,1,0,1,0,0,0,...,0,12,4,2011,6,5,10,4,2009,764
4,41,87mebub9p4,1,0,0,0,0,1,0,0,...,1,9,3,2010,6,1,12,4,2009,279


## Summary
### Changed training dataset from 15 columns to 164 columns
#### HW: Investigate what information could be extracted from the other non-date columns?
#### HW: Check what external data could be added? 


### HW: With dummy variables created by One-hot-encoding are we introducing multicollinearity ?
#### If yes... can we use PCA to reduce the dimensionality down to prevent the model from being misled?
### HW: How worried about multicollinearity you need to be if you are (i) building a regression model and (ii) concerned with prediction accuracy ?
#### StackExchange multicollinearity: https://stats.stackexchange.com/questions/168622/why-is-multicollinearity-not-checked-in-modern-statistics-machine-learning


## What Next? - Adding new data
## Milestone: Understanding the sessions.csv data.

In [18]:
## Loading sessions.csv data
print("Reading sessions data...")
sessions_file = "./traveler/sessions.csv"
df_sessions = pd.read_csv(sessions_file, header = 0,index_col=False)
print("Reading sessions data...completed")
df_sessions.head()

Reading sessions data...
Reading sessions data...completed


,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435


In [18]:
df_sessions.shape[0]


10567737

### Note:
   #### While combining sessions data with the training data, we need to aggregate the sessions data so that there is one row per user (as opposed to many rows for each user)
   #### Two fields are striking - device_type and secs_elapsed
   ##### These fields provide important information that could help to more accurately predict which country a user will make a first booking in. 
   ##### For example, imagine that people spending relatively little time to make a booking on a phone are likely to be making bookings in locations closer to home (i.e. the US) than someone spending more time to make a booking on a desktop computer. 
   ##### Remember this is just a theory that needs to be proven, but it is a good reason to ensure it helps in final training dataset
    

## Cleaning and Transforming the Data
### Goal: remember we need to get the final data into a format that can be merged with the data created for df_all (i.e., train + test data)



### [1] Extract the primary and secondary devices for each user
##### How do we determine what a user’s primary and secondary devices are?
##### Solution: Look at how much time user's spent on each device
<img src="./images/primary_secondary_device_type.jpg" height="400" width="500"/>

#### Note: By aggregating the data this way, we are also implicitly removing the missing values.
#### Hint: ref Pandas Groupby 


In [19]:
# Determine primary device
print("Determing primary device...")
sessions_device = df_sessions.loc[:, ['user_id', 'device_type', 
                                      'secs_elapsed']]
aggregated_lvl1 = sessions_device.groupby(['user_id', 'device_type'], 
                                          as_index=False, 
                                          sort=False).aggregate(np.sum)
#aggregated_lvl1.head(10)
idx = aggregated_lvl1.groupby(['user_id'], sort=False)['secs_elapsed']
.transform(max) == aggregated_lvl1['secs_elapsed']
#idx.head(10)
df_sessions_primary = pd.DataFrame(aggregated_lvl1.loc[idx , 
                                ['user_id', 'device_type', 'secs_elapsed']])
#df_sessions_primary.head(10)
df_sessions_primary.rename(columns = {'device_type':'primary_device', 
                        'secs_elapsed':'primary_secs'}, inplace=True)
#df_sessions_primary.head(10)
# Call user defined One Hot Encoding function
df_sessions_primary = convert_to_binary(df=df_sessions_primary, 
                                        column_to_convert='primary_device')
#df_sessions_primary.head()
df_sessions_primary.drop('primary_device', axis=1, inplace=True)
#df_sessions_primary.head()
print("Determing primary device...completed")

Determing primary device...
Determing primary device...completed


In [20]:
## Testing ...
df_sessions_primary.head()

,user_id,primary_secs,prima_windows_de,prima_mac_deskto,prima_iphone,prima_ipad_table,prima_unknown,prima_android_ap,prima_linux_desk,prima_tablet,prima_chromebook,prima_android_ph,prima_ipodtouch,prima_blackberry,prima_windows_ph,prima_opera_phon
0,d1mm9tcy42,3315820,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,yo8nz8bqcq,207842,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,4grx6yxeby,1059362,1,0,0,0,0,0,0,0,0,0,0,0,0,0
5,ncf87guaf0,3361226,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,4rvqpxoh3h,2555,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [21]:
# Determine Secondary device
print("Determing secondary device...")
remaining = aggregated_lvl1.drop(aggregated_lvl1.index[idx])
remaining.head()
idx = remaining.groupby(['user_id'], sort=False)['secs_elapsed'].transform(max) == remaining['secs_elapsed']
df_sessions_secondary = pd.DataFrame(remaining.loc[idx , ['user_id', 'device_type', 'secs_elapsed']])
df_sessions_secondary.rename(columns = {'device_type':'secondary_device', 'secs_elapsed':'secondary_secs'}, inplace=True)
df_sessions_secondary = convert_to_binary(df=df_sessions_secondary, column_to_convert='secondary_device')
df_sessions_secondary.drop('secondary_device', axis=1, inplace=True)
print("Determing secondary device...completed")

Determing secondary device...
Determing secondary device...completed


In [22]:
## Testing ...
df_sessions_secondary.head()

,user_id,secondary_secs,secon_unknown,secon_android_ph,secon_ipad_table,secon_android_ap,secon_mac_deskto,secon_iphone,secon_windows_de,secon_linux_desk,secon_tablet,secon_blackberry,secon_windows_ph,secon_chromebook,secon_opera_phon,secon_ipodtouch
1,d1mm9tcy42,111709,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4grx6yxeby,76082,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7,ncf87guaf0,274002,0,1,0,0,0,0,0,0,0,0,0,0,0,0
11,xwxei6hdk4,810,0,0,1,0,0,0,0,0,0,0,0,0,0,0
15,awiurksqr3,488,0,0,1,0,0,0,0,0,0,0,0,0,0,0


### [2] Determine Counts of Actions
#### What? Counts of how many times each action was taken by each user
#### Solution: Two-step process
#### Step1: determine the count of each action for each user
<img src="./images/Count_of_actions_Step1.jpg" height="400" width="500"/>

#### Step2: pivot table
<img src="./images/Count_of_actions_Step2.jpg" height="400" width="500"/>

### [3] Looping Through the Actions Columns
#### Problem: Transformation works for one action column at a time, but in the data we have three action columns: action, action_type and action_detail
#### Solution: Repeat above steps for each column individually, effectively creating three separate tables.
#### Effective Approach: Since this now create tables where each row represents one user, JOIN (SQL concept) these three tables together on the basis of the user id.


In [27]:
# Count occurrences of value in a column
def convert_to_counts(df, id_col, column_to_convert):
    id_list = df[id_col].drop_duplicates()
    id_list.head()
    df_counts = df.loc[:,[id_col, column_to_convert]]
    df_counts['count'] = 1
    df_counts = df_counts.groupby(by=[id_col, column_to_convert], as_index=False, sort=False).sum()

    new_df = df_counts.pivot(index=id_col, columns=column_to_convert, values='count')
    new_df = new_df.fillna(0)

# Rename Columns
    categories = list(df[column_to_convert].drop_duplicates())
    for category in categories:
        cat_name = str(category).replace(" ", "_").replace("(", "").replace(")", "").replace("/", "_").replace("-", "").lower()
        col_name = column_to_convert + '_' + cat_name
        new_df.rename(columns = {category:col_name}, inplace=True)

    return new_df

# Aggregate and combine actions taken columns
print("Aggregating actions taken...")
session_actions = df_sessions.loc[:,['user_id', 'action', 'action_type', 'action_detail']]
columns_to_convert = ['action', 'action_type', 'action_detail']
session_actions = session_actions.fillna('not provided')
first = True

for column in columns_to_convert:
    print("Converting " + column + " column...")
    current_data = convert_to_counts(df=session_actions, id_col='user_id', column_to_convert=column)

# If first loop, current data becomes existing data, otherwise merge existing and current
if first:
    first = False
    actions_data = current_data
else:
    actions_data = pd.concat([actions_data, current_data], axis=1, join='inner')


Aggregating actions taken...
Converting action column...
Converting action_type column...
Converting action_detail column...


In [24]:
##Testing...
actions_data.head()

action_detail,action_detail_unknown,action_detail_account_notification_settings,action_detail_account_payment_methods,action_detail_account_payout_preferences,action_detail_account_privacy_settings,action_detail_account_transaction_history,action_detail_admin_templates,action_detail_airbnb_picks_wishlists,action_detail_alteration_field,action_detail_alteration_request,...,action_detail_view_resolutions,action_detail_view_search_results,action_detail_view_security_checks,action_detail_view_user_real_names,action_detail_wishlist,action_detail_wishlist_content_update,action_detail_wishlist_note,action_detail_your_listings,action_detail_your_reservations,action_detail_your_trips
user_id,,,,,,,,,,,,,,,,,,,,,
00023iyk9l,0,0,0,0,0,0,0,0,0,0,...,0,5,0,0,0,4,0,0,0,2
0010k6l0om,5,0,0,0,0,0,0,0,0,0,...,0,10,0,0,0,8,0,0,0,0
001wyh0pz8,6,0,0,0,0,0,0,0,0,0,...,0,66,0,0,0,0,0,0,0,0
0028jgx1x1,1,0,0,0,0,0,0,0,0,0,...,0,9,0,0,0,0,0,0,0,0
002qnbzfs5,184,0,0,0,0,0,0,0,0,0,...,0,125,0,0,0,0,0,0,0,0


### [4] Finally, Combine Data Sets (Data Integration)
#### [4.1] First, combine the two device dataframes (df_primary and df_secondary) to create a device dataframe. 
#### [4.2] Then, combine the device dataframe with the actions dataframe to create a sessions dataframe with all the features extracted from sessions.csv
#### [4.3] Finally, combine the sessions dataframe with the users(train+test) dataframe computed earlier

In [25]:
# [4.1] Merge device datasets
print("Combining results...")
df_sessions_primary.set_index('user_id', inplace=True)
df_sessions_secondary.set_index('user_id', inplace=True)
device_data = pd.concat([df_sessions_primary, df_sessions_secondary], axis=1, join="outer")

# [4.2] Merge device and actions datasets
combined_results = pd.concat([device_data, actions_data], axis=1, join='outer')
df_sessions_complete = combined_results.fillna(0)

# [4.3] Merge user and session datasets
df_all.set_index('id', inplace=True)
df_all = pd.concat([df_all, df_sessions_complete], axis=1, join='inner')
print("Combining results...completed")


Combining results...
Combining results...completed


In [26]:
df_all.head()

,age,gende_unknown,gende_male,gende_female,gende_other,signu_facebook,signu_basic,signu_google,signu_weibo,signu_0,...,action_detail_view_resolutions,action_detail_view_search_results,action_detail_view_security_checks,action_detail_view_user_real_names,action_detail_wishlist,action_detail_wishlist_content_update,action_detail_wishlist_note,action_detail_your_listings,action_detail_your_reservations,action_detail_your_trips
00023iyk9l,31,1,0,0,0,0,1,0,0,1,...,0,5,0,0,0,4,0,0,0,2
0010k6l0om,-1,1,0,0,0,0,1,0,0,1,...,0,10,0,0,0,8,0,0,0,0
001wyh0pz8,-1,1,0,0,0,0,1,0,0,0,...,0,66,0,0,0,0,0,0,0,0
0028jgx1x1,-1,1,0,0,0,0,1,0,0,1,...,0,9,0,0,0,0,0,0,0,0
002qnbzfs5,26,0,0,1,0,1,0,0,0,0,...,0,125,0,0,0,0,0,0,0,0


### A Note on Joins

#### [*] 4.1 requires outer join because not all users have a secondary device. Doing an outer join here ensures that the dataset includes all users regardless of this fact.
#### [*] 4.2 requires outer join just to ensure that if a user is missing from one of the datasets (for whatever reason), we will still capture them. Fill any missing values with 0s to ensure not to have any NULL values that may have been generated by these outer joins.
#### [*] 4.3 requires inner join i.e., want to have final training dataset to only include users that also have sessions data (filters others).


## Next Session: Prepare a learning model to make predictions.
    